In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics opencv-python-headless torch torchvision transformers pillow opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 11.8 MB/s eta 0:00:00


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from torchvision import transforms
from PIL import Image
import cv2
import torch
import glob
from tqdm import tqdm
import os

In [9]:
# 테스트 코드
yolo_model_path = '/content/drive/MyDrive/project/KDT_kiosk/model/pre_yolov8face/yolov8n-face.pt'
# YOLOv8 모델 로드
yolo_model = YOLO(yolo_model_path)

In [31]:
# # yolo 결과가 어떤형태인지 보기위해서 만든 코드

# # 입력 이미지와 출력 폴더 경로
# image_folders = [
#     '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_test',
#     '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_train'
# ]
# output_folders = [
#     '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_test_cropped',
#     '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_train_cropped'
# ]

# # 각 폴더에 대해 이미지 처리
# for i in range(1):
#     image_folder = image_folders[i]
#     output_folder = output_folders[i]

#     # 이미지 파일 리스트
#     image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

#     # 파일 하나씩 처리
#     for image_file in image_files[1]:
#         image_path = os.path.join(image_folder, image_file)

#         # 이미지 읽기
#         img = cv2.imread(image_path)

#         # YOLOv8 모델을 사용하여 얼굴 감지
#         results = yolo_model(img, conf=0.25, imgsz=1280, max_det=1000)

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 1280x960 2 faces, 1388.6ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 736x1280 2 faces, 893.1ms
Speed: 21.5ms preprocess, 1140.9ms inference, 2.1ms postprocess per image at shape (1, 3, 736, 1280)
WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 1280x960 2 faces, 1049.5ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 736x1280 2 faces, 847.8ms
Speed: 17.9ms preprocess, 948.7ms inference, 2.6ms postprocess per image at shape (1, 3, 736, 1280)
WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/u

In [29]:
## yolo 출력 형태 확인

# results[0].boxes.xyxy
## 결과가 2개 박스가 나왔다... 이때 눈치 챘어야했는데

tensor([[115., 417., 155., 469.],
        [271., 421., 307., 471.]])

In [38]:
# UTK_test, UTK_train 폴더에 있는 이미지를 YOLO에 하나씩 넣어서,
# 얼굴을 크롭한 뒤 UTK_test_cropped, UTK_train_cropped 폴더에 저장

# 입력 이미지와 출력 폴더 경로
image_folders = [
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_test',
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_train'
]
output_folders = [
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_test_cropped',
    '/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_train_cropped'
]

# 각 폴더에 대해 이미지 처리
for i in range(2):
    image_folder = image_folders[i]
    output_folder = output_folders[i]

    # 출력 폴더가 존재하지 않으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 이미지 파일 리스트
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # 파일 하나씩 처리
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)

        # 이미지 읽기
        img = cv2.imread(image_path)

        # YOLOv8 모델을 사용하여 얼굴 감지
        results = yolo_model(img, conf=0.25, imgsz=1280, max_det=1000)
        # conf 값이 높을 수록 신뢰도 임계점이 높아져서 신뢰도가 높은 것만 반환
        # max_det=1 로 조정하면 결과 중 신뢰도가 가장 높은 1개만 남긴다

        # 감지된 박스 좌표 가져오기
        boxes = results[0].boxes.xyxy.cpu().numpy()  # 감지된 박스의 좌표를 NumPy 배열로 변환

        # 결과를 처리하고 크롭된 얼굴 저장
        for i, box in tqdm(enumerate(boxes)): # confidence에 조건문으로 기준 주면 좀 더 신뢰도가 높은 얼굴만 선정된다
            x_min, y_min, x_max, y_max = box.astype(int)

            # 얼굴 영역 크롭
            cropped_face = img[y_min:y_max, x_min:x_max]

            # 크롭된 얼굴 이미지 저장
            output_path = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_face_{i}.jpg")
            cv2.imwrite(output_path, cropped_face)

        print(f'Processed {image_file} and saved cropped faces to {output_folder}')


In [36]:
# # _0.jpg으로 끝나지 않는 이미지 모두 삭제

# directory = ["/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_test_cropped",
#              "/content/drive/MyDrive/project/KDT_kiosk/data/est_face_emotion/face_crop/UTK_train_cropped"]

# # 디렉토리 내의 모든 파일을 순회
# for folder in directory:
#   for filename in tqdm(os.listdir(folder)):
#       # 파일 이름이 '_0.jpg'로 끝나지 않으면
#       if not filename.endswith("_0.jpg"):
#           # 파일 전체 경로 생성
#           full_file_path = os.path.join(folder, filename)
#           os.remove(full_file_path)

100%|██████████| 26038/26038 [00:48<00:00, 532.95it/s]
